# SAR Imagery and AIS Data Integration

This notebook demonstrates how to download Sentinel-1 SAR (Synthetic Aperture Radar) imagery and corresponding AIS (Automatic Identification System) data for maritime monitoring applications.

## Use Case
Combining SAR imagery with AIS data allows for comprehensive maritime surveillance:
- **SAR imagery** provides weather-independent detection of vessels and ocean features
- **AIS data** provides vessel identification, position, and navigation information
- **Integration** enables vessel verification, dark vessel detection, and maritime traffic analysis

## Area of Interest
We'll focus on the **North Sea shipping lanes** near the Netherlands coast, a busy maritime area with high vessel traffic.

## 1. Setup and Configuration

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from datetime import datetime, date, time
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Import phidown modules
from phidown import CopernicusDataSearcher
from phidown.ais import download_ais_data, AISDataHandler
from phidown.viz import plot_kml_coordinates

# Optional: Import interactive tools if available
try:
    from phidown.interactive_tools import create_polygon_tool
    INTERACTIVE_AVAILABLE = True
except ImportError:
    INTERACTIVE_AVAILABLE = False
    print("Interactive tools not available. Install with: pip install phidown[viz]")

# Set style for plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## 2. Define Area of Interest and Time Period

In [ ]:
# Define the Area of Interest (AOI) - North Sea shipping lanes
# This area covers busy shipping routes near the Netherlands coast
aoi_wkt = """POLYGON((
    3.0 52.0,
    5.5 52.0,
    5.5 53.5,
    3.0 53.5,
    3.0 52.0
))"""

# Define time period for analysis
target_date = "2024-08-15"  # Summer date with good weather conditions
start_time = "06:00:00"     # Morning time to capture commuter and commercial traffic
end_time = "18:00:00"       # Evening time

print(f"Analysis Configuration:")
print(f"  Area: North Sea shipping lanes (Netherlands coast)")
print(f"  Date: {target_date}")
print(f"  Time window: {start_time} - {end_time} UTC")
print(f"  AOI: {aoi_wkt.strip()}")

## 3. Search for Sentinel-1 SAR Data

In [ ]:
# Initialize Copernicus data searcher
searcher = CopernicusDataSearcher()

# Search for Sentinel-1 GRD products
print("Searching for Sentinel-1 SAR data...")
sar_results = searcher.search(
    collection_name='SENTINEL-1',
    aoi_wkt=aoi_wkt,
    start_date=target_date,
    end_date=target_date,
    product_type='GRD',  # Ground Range Detected - most common SAR product
    sensor_mode='IW',    # Interferometric Wide swath mode
    polarisation='VV VH' # Dual polarization for better vessel detection
)

print(f"Found {len(sar_results)} Sentinel-1 products")

# Display search results
if len(sar_results) > 0:
    # Select relevant columns for display
    display_cols = ['Name', 'ContentDate', 'CloudCover', 'Instrument', 'ProductType', 'Polarisation']
    available_cols = [col for col in display_cols if col in sar_results.columns]
    
    print("\nSAR Data Products:")
    print(sar_results[available_cols].head())
    
    # Get the first product for download
    selected_sar = sar_results.iloc[0]
    print(f"\nSelected SAR product: {selected_sar.get('Name', 'Unknown')}")
    print(f"Acquisition time: {selected_sar.get('ContentDate', 'Unknown')}")
else:
    print("No SAR data found for the specified criteria.")
    print("Try adjusting the date or area of interest.")

## 4. Download AIS Data

In [ ]:
# Download AIS data for the same area and time period
print("Downloading AIS data...")

try:
    ais_data = download_ais_data(
        start_date=target_date,
        start_time=start_time,
        end_time=end_time,
        aoi_wkt=aoi_wkt
    )
    
    print(f"Downloaded {len(ais_data)} AIS records")
    
    if len(ais_data) > 0:
        print("\nAIS Data Sample:")
        print(ais_data.head())
        
        # Basic statistics
        print(f"\nAIS Data Statistics:")
        print(f"  Total vessels tracked: {ais_data['mmsi'].nunique()}")
        print(f"  Named vessels: {len(ais_data[ais_data['name'] != ''])}")
        print(f"  Latitude range: {ais_data['lat'].min():.3f} to {ais_data['lat'].max():.3f}")
        print(f"  Longitude range: {ais_data['lon'].min():.3f} to {ais_data['lon'].max():.3f}")
        
        # Temporal distribution
        ais_data['hour'] = pd.to_datetime(ais_data['timestamp']).dt.hour
        hourly_counts = ais_data['hour'].value_counts().sort_index()
        print(f"\nVessel activity by hour:")
        for hour, count in hourly_counts.items():
            print(f"  {hour:02d}:00 - {count} records")
    else:
        print("No AIS data found for the specified criteria.")
        
except Exception as e:
    print(f"Error downloading AIS data: {e}")
    print("Note: AIS data requires additional dependencies. Install with: pip install phidown[ais]")
    ais_data = pd.DataFrame()  # Empty dataframe for continuation

## 5. Spatial Analysis and Visualization

In [ ]:
# Create visualization of vessel positions
if len(ais_data) > 0:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot 1: Vessel positions
    scatter = ax1.scatter(ais_data['lon'], ais_data['lat'], 
                         c=ais_data['hour'], cmap='viridis', 
                         alpha=0.6, s=20)
    ax1.set_xlabel('Longitude')
    ax1.set_ylabel('Latitude')
    ax1.set_title('Vessel Positions Colored by Hour')
    ax1.grid(True, alpha=0.3)
    
    # Add colorbar
    cbar = plt.colorbar(scatter, ax=ax1)
    cbar.set_label('Hour of Day')
    
    # Plot 2: Vessel activity by hour
    hourly_counts.plot(kind='bar', ax=ax2, color='skyblue')
    ax2.set_xlabel('Hour of Day')
    ax2.set_ylabel('Number of AIS Records')
    ax2.set_title('Vessel Activity by Hour')
    ax2.tick_params(axis='x', rotation=45)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Vessel density analysis
    print("\nSpatial Analysis:")
    
    # Create a simple grid for density analysis
    lat_bins = np.linspace(ais_data['lat'].min(), ais_data['lat'].max(), 10)
    lon_bins = np.linspace(ais_data['lon'].min(), ais_data['lon'].max(), 10)
    
    # Count vessels in each grid cell
    hist, _, _ = np.histogram2d(ais_data['lat'], ais_data['lon'], bins=[lat_bins, lon_bins])
    
    # Find the busiest area
    max_density_idx = np.unravel_index(np.argmax(hist), hist.shape)
    busiest_lat = (lat_bins[max_density_idx[0]] + lat_bins[max_density_idx[0] + 1]) / 2
    busiest_lon = (lon_bins[max_density_idx[1]] + lon_bins[max_density_idx[1] + 1]) / 2
    
    print(f"  Busiest shipping area: {busiest_lat:.3f}°N, {busiest_lon:.3f}°E")
    print(f"  Max vessel density: {hist.max():.0f} vessels per grid cell")
else:
    print("No AIS data available for visualization.")

## 6. Data Integration Analysis

In [ ]:
# Analyze the relationship between SAR acquisition time and AIS data
if len(sar_results) > 0 and len(ais_data) > 0:
    # Extract SAR acquisition time
    sar_time_str = selected_sar.get('ContentDate', '')
    if sar_time_str:
        # Parse the SAR acquisition time
        try:
            # Handle different possible datetime formats
            if 'T' in sar_time_str:
                sar_datetime = pd.to_datetime(sar_time_str.split('T')[0] + ' ' + 
                                            sar_time_str.split('T')[1].split('.')[0])
            else:
                sar_datetime = pd.to_datetime(sar_time_str)
            
            sar_hour = sar_datetime.hour
            
            print(f"SAR/AIS Integration Analysis:")
            print(f"  SAR acquisition time: {sar_datetime}")
            print(f"  SAR acquisition hour: {sar_hour:02d}:00")
            
            # Find AIS records near the SAR acquisition time
            time_window = 1  # ±1 hour window
            ais_near_sar = ais_data[
                (ais_data['hour'] >= sar_hour - time_window) & 
                (ais_data['hour'] <= sar_hour + time_window)
            ]
            
            print(f"  AIS records within ±{time_window}h of SAR: {len(ais_near_sar)}")
            print(f"  Unique vessels near SAR time: {ais_near_sar['mmsi'].nunique()}")
            
            if len(ais_near_sar) > 0:
                print(f"\nVessels Active During SAR Acquisition:")
                vessel_summary = ais_near_sar.groupby('mmsi').agg({
                    'name': 'first',
                    'lat': ['min', 'max', 'mean'],
                    'lon': ['min', 'max', 'mean'],
                    'timestamp': 'count'
                }).round(4)
                
                vessel_summary.columns = ['Name', 'Min_Lat', 'Max_Lat', 'Avg_Lat', 
                                        'Min_Lon', 'Max_Lon', 'Avg_Lon', 'Records']
                
                print(vessel_summary.head(10))
                
        except Exception as e:
            print(f"Error parsing SAR acquisition time: {e}")
    
    # Calculate potential vessel detection opportunities
    print(f"\nVessel Detection Opportunities:")
    total_vessels = ais_data['mmsi'].nunique()
    named_vessels = len(ais_data[ais_data['name'] != '']['mmsi'].unique())
    
    print(f"  Total unique vessels in AIS data: {total_vessels}")
    print(f"  Named vessels (easier to verify): {named_vessels}")
    print(f"  Anonymous vessels: {total_vessels - named_vessels}")
    
    # Vessel size estimation (simplified)
    vessel_tracks = ais_data.groupby('mmsi').size()
    active_vessels = vessel_tracks[vessel_tracks >= 5]  # Vessels with 5+ position reports
    
    print(f"  Active vessels (5+ reports): {len(active_vessels)}")
    print(f"  Average reports per vessel: {vessel_tracks.mean():.1f}")

else:
    print("Integration analysis requires both SAR and AIS data.")

## 7. Export and Save Data

In [ ]:
# Create output directory
output_dir = Path("sar_ais_analysis_output")
output_dir.mkdir(exist_ok=True)

# Save SAR search results
if len(sar_results) > 0:
    sar_output_file = output_dir / f"sar_search_results_{target_date}.csv"
    sar_results.to_csv(sar_output_file, index=False)
    print(f"SAR search results saved to: {sar_output_file}")

# Save AIS data
if len(ais_data) > 0:
    ais_output_file = output_dir / f"ais_data_{target_date}.csv"
    ais_data.to_csv(ais_output_file, index=False)
    print(f"AIS data saved to: {ais_output_file}")
    
    # Save vessel summary
    vessel_summary_file = output_dir / f"vessel_summary_{target_date}.csv"
    vessel_summary = ais_data.groupby('mmsi').agg({
        'name': 'first',
        'lat': ['min', 'max', 'mean', 'count'],
        'lon': ['min', 'max', 'mean'],
        'source_date': 'first'
    }).round(6)
    
    vessel_summary.columns = ['Vessel_Name', 'Min_Lat', 'Max_Lat', 'Avg_Lat', 'Position_Count',
                            'Min_Lon', 'Max_Lon', 'Avg_Lon', 'Date']
    vessel_summary.to_csv(vessel_summary_file)
    print(f"Vessel summary saved to: {vessel_summary_file}")

# Create analysis summary
summary_file = output_dir / f"analysis_summary_{target_date}.txt"
with open(summary_file, 'w') as f:
    f.write(f"SAR and AIS Integration Analysis Summary\n")
    f.write(f"======================================\n\n")
    f.write(f"Analysis Date: {target_date}\n")
    f.write(f"Time Window: {start_time} - {end_time} UTC\n")
    f.write(f"Area of Interest: North Sea shipping lanes\n")
    f.write(f"AOI Coordinates: {aoi_wkt.strip()}\n\n")
    
    f.write(f"SAR Data Results:\n")
    f.write(f"  Products found: {len(sar_results)}\n")
    if len(sar_results) > 0:
        f.write(f"  Selected product: {selected_sar.get('Name', 'Unknown')}\n")
        f.write(f"  Acquisition time: {selected_sar.get('ContentDate', 'Unknown')}\n")
    
    f.write(f"\nAIS Data Results:\n")
    f.write(f"  Total records: {len(ais_data)}\n")
    if len(ais_data) > 0:
        f.write(f"  Unique vessels: {ais_data['mmsi'].nunique()}\n")
        f.write(f"  Named vessels: {len(ais_data[ais_data['name'] != ''])}\n")

print(f"\nAnalysis summary saved to: {summary_file}")
print(f"All output files are in: {output_dir.absolute()}")

## 8. Next Steps and Applications

This notebook demonstrated the integration of SAR imagery and AIS data for maritime monitoring. Here are some potential next steps:

### Data Download and Processing
1. **Download SAR data**: Use the search results to download actual SAR imagery
2. **Process SAR imagery**: Apply speckle filtering, calibration, and vessel detection algorithms
3. **Temporal correlation**: Match vessel positions from AIS with SAR acquisition times

### Advanced Analysis
1. **Vessel Detection**: Use SAR imagery to detect vessels and compare with AIS positions
2. **Dark Vessel Detection**: Identify vessels visible in SAR but missing from AIS
3. **Route Analysis**: Analyze vessel movement patterns and shipping lanes
4. **Environmental Monitoring**: Combine with weather data for maritime safety analysis

### Applications
- **Maritime Security**: Illegal fishing detection, border monitoring
- **Traffic Management**: Port optimization, collision avoidance
- **Environmental Protection**: Oil spill monitoring, protected area surveillance
- **Search and Rescue**: Emergency response coordination

### Technical Improvements
- Implement automated vessel detection algorithms
- Add real-time data processing capabilities
- Integrate with additional data sources (weather, port schedules)
- Develop machine learning models for vessel classification